In [ ]:
import numpy as np
import pandas as pd

In [ ]:
conditions = pd.read_csv("../input/the-depression-dataset/data/scores.csv")

In [ ]:
conditions = conditions[conditions.number.str.startswith("condition")]
conditions

In [ ]:
import os
sum_dict = {}
for file in os.listdir("../input/the-depression-dataset/data/condition"):
    df = pd.read_csv("../input/the-depression-dataset/data/condition" + "/" + file)
    sum_dict[file.split(".")[0]] = df.activity.sum()
sum_dict

In [ ]:
sums = pd.DataFrame(pd.Series(sum_dict))
sums.columns = ["Sum"]
sums

In [ ]:
conditions = conditions.set_index("number").join(sums).reset_index()

In [ ]:
conditions

In [ ]:
conditions.at[21, "edu"] = "6-10"
conditions

In [ ]:
conditions["Stress"] = conditions.Sum / conditions.days
conditions

In [ ]:
conditions.melanch.fillna(2.0, inplace = True)
conditions

In [ ]:
conditions = conditions.drop(columns = ["number", "Sum", "days"])
conditions

In [ ]:
list(conditions.columns)

In [ ]:
categoricals = list(conditions.columns)[:-3]
categoricals

In [ ]:
for c in categoricals:
    conditions[c] = conditions[c].astype("category").cat.codes
conditions

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler().fit(conditions.Stress.values.reshape(-1,1))

In [ ]:
conditions.Stress = sc.fit_transform(conditions.Stress.values.reshape(-1,1))
conditions

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')


for c in categoricals:
    enc_df = pd.DataFrame(enc.fit_transform(conditions[[c]]).toarray())
    conditions = conditions.drop(columns = c).join(enc_df, lsuffix = c[0])
conditions

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor()
reg.fit(conditions.drop(columns = "Stress"), conditions.Stress)

In [ ]:
from sklearn.metrics import r2_score
r2_score(conditions.Stress, reg.predict(conditions.drop(columns = "Stress")))